In [1]:


import math
import csv
import cv2
from math import pi as PI
import numpy as np

#sources
#superconductors source dataset. it has 21263 rows
cs_materials=open('source_data/unique_m.csv', newline='')
materialreader = csv.reader(cs_materials, delimiter=',', quotechar='|')

#periodic chemestry table
cs_pt=open('source_data/PTE.csv', newline='')
ptreader = csv.reader(cs_pt, delimiter=',', quotechar='|')

#target files
#in this file will be train dataset. it will have 95680 rows(19136*5)(90% of all dataset). 
traincswriter=open('processed_data/train_dataset.txt', 'a', newline='')
trainwriter = csv.writer(traincswriter, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

#in this file will be test dataset.it will have 10635 rows(2127*5)(10% of all dataset)
testcswriter=open('processed_data/test_dataset.txt', 'a', newline='')
testwriter = csv.writer(testcswriter, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)


#here will be processed chemestry table
PTECwriter=open('processed_data/PTE_adapted.txt', 'a', newline='')
PTECUTEDwriter = csv.writer(PTECwriter, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)


In [2]:
#skip headers of periodic tadble
h_ptrow = next(ptreader)
print(h_ptrow)

['AtomicNumber', 'Element', 'Symbol', 'AtomicMass', 'NumberofNeutrons', 'NumberofProtons', 'NumberofElectrons', 'Period', 'Group', 'Phase', 'Radioactive', 'Natural', 'Metal', 'Nonmetal', 'Metalloid', 'Type', 'AtomicRadius', 'Electronegativity', 'FirstIonization', 'Density', 'MeltingPoint', 'BoilingPoint', 'NumberOfIsotopes', 'Discoverer', 'Year', 'SpecificHeat', 'NumberofShells', 'NumberofValence']


In [3]:
#find indexes of target parametrs
for i in range(len(h_ptrow)):
    print(str(i)+"  "+h_ptrow[i])

0  AtomicNumber
1  Element
2  Symbol
3  AtomicMass
4  NumberofNeutrons
5  NumberofProtons
6  NumberofElectrons
7  Period
8  Group
9  Phase
10  Radioactive
11  Natural
12  Metal
13  Nonmetal
14  Metalloid
15  Type
16  AtomicRadius
17  Electronegativity
18  FirstIonization
19  Density
20  MeltingPoint
21  BoilingPoint
22  NumberOfIsotopes
23  Discoverer
24  Year
25  SpecificHeat
26  NumberofShells
27  NumberofValence


In [4]:
#read chemestry table until Radon
#radon is last element in superconductors dataset table unique_m
pt_counter=1
pt_list=[]

while(pt_counter<87):
    pt_counter=pt_counter+1
    ptrow = next(ptreader)
    pt_list.append(ptrow)
    print(str(ptrow))
    

['1', 'Hydrogen', 'H', '1.007', '0', '1', '1', '1', '1', 'gas', '', 'yes', '', 'yes', '', 'Nonmetal', '0.79', '2.2', '13.5984', '8.99E-05', '14.175', '20.28', '3', 'Cavendish', '1766', '14.304', '1', '1']
['2', 'Helium', 'He', '4.002', '2', '2', '2', '1', '18', 'gas', '', 'yes', '', 'yes', '', 'Noble Gas', '0.49', '', '24.5874', '1.79E-04', '', '4.22', '5', 'Janssen', '1868', '5.193', '1', '']
['3', 'Lithium', 'Li', '6.941', '4', '3', '3', '2', '1', 'solid', '', 'yes', 'yes', '', '', 'Alkali Metal', '2.1', '0.98', '5.3917', '5.34E-01', '453.85', '1615', '5', 'Arfvedson', '1817', '3.582', '2', '1']
['4', 'Beryllium', 'Be', '9.012', '5', '4', '4', '2', '2', 'solid', '', 'yes', 'yes', '', '', 'Alkaline Earth Metal', '1.4', '1.57', '9.3227', '1.85E+00', '1560.15', '2742', '6', 'Vaulquelin', '1798', '1.825', '2', '2']
['5', 'Boron', 'B', '10.811', '6', '5', '5', '2', '13', 'solid', '', 'yes', '', '', 'yes', 'Metalloid', '1.2', '2.04', '8.298', '2.34E+00', '2573.15', '4200', '6', 'Gay-Lussac

In [5]:
#select parametrs of chemestry element
#most important

#now used: 3,4,6,7,8,12,13,14, 16,17,18,19,20,21,25,26. Parametrs 12,13,14 has one hot encoding format
pt_counter=0
copy=1
params_order=[3,4,6,7,8,16,17,18,19,20,21,25,26]
one_hot_params_order=[12,13,14]
clean_pt_list=[]


while(pt_counter<86):
    current_row=[]
    for i in range(27):
        for j in range(len(params_order)):
            if params_order[j]==i:
                if pt_list[pt_counter][i] != "":
                    current_row.append(float(pt_list[pt_counter][i]))
                else:
                    current_row.append(float(0))
        
     
    for i in range(27):        
         for j in range(len(one_hot_params_order)):
            if one_hot_params_order[j]==i:
                if str(pt_list[pt_counter][i])=="yes":
                    current_row.append(float(1))
                else:
                    current_row.append(float(0))
        
    clean_pt_list.append(current_row)
    pt_counter=pt_counter+1

print("number of parametrs per element")
print(len(clean_pt_list[85]))
num_params=len(clean_pt_list[85])
print("----------------------")
#example for element
for i in range(num_params):
    print(str(i)+"   "+str(clean_pt_list[85][i]))


number of parametrs per element
16
----------------------
0   222.0
1   136.0
2   86.0
3   6.0
4   18.0
5   1.3
6   0.0
7   10.7485
8   0.00973
9   202.15
10   211.3
11   0.094
12   6.0
13   1.0
14   0.0
15   0.0


In [6]:
#normalization of element parametrs

#mean values for every parametr
mean_clean_pt_list=np.asarray(clean_pt_list)
mean = mean_clean_pt_list.mean(axis=0)
print(mean)
print("---------------------------")
#standard deviation for every parametr
std = mean_clean_pt_list.std(axis=0)
print(std)
print("----------------------")
#normalization
mean_clean_pt_list = mean_clean_pt_list-mean
mean_clean_pt_list /= std
mean_clean_pt_list=mean_clean_pt_list.tolist()
print("if all is correctly values are euqal")
print(len(clean_pt_list))
print(len(mean_clean_pt_list))
print("----------------------")
print("if all is correctly values are euqal")
print(len(clean_pt_list[85]))
print(len(mean_clean_pt_list[85]))
print("----------------------")
print("source and normalized parametrs of element")
print(clean_pt_list[85])
print(mean_clean_pt_list[85])




[1.03889721e+02 6.03837209e+01 4.35000000e+01 4.60465116e+00
 8.41860465e+00 1.82581395e+00 1.66046512e+00 8.36080000e+00
 6.85821964e+00 1.26663509e+03 2.44068640e+03 2.12540930e+01
 4.56370930e+00 7.20930233e-01 1.97674419e-01 8.13953488e-02]
---------------------------
[6.37807181e+01 3.89830520e+01 2.48243832e+01 1.41650626e+00
 6.36572420e+00 6.07494789e-01 7.62416457e-01 3.47854693e+00
 5.51930872e+00 9.53228900e+02 1.60842111e+03 1.90119016e+02
 1.48272760e+00 4.48541896e-01 3.98245204e-01 2.73441303e-01]
----------------------
if all is correctly values are euqal
86
86
----------------------
if all is correctly values are euqal
16
16
----------------------
source and normalized parametrs of element
[222.0, 136.0, 86.0, 6.0, 18.0, 1.3, 0.0, 10.7485, 0.00973, 202.15, 211.3, 0.094, 6.0, 1.0, 0.0, 0.0]
[1.8518179563894672, 1.9397218862988037, 1.7120264258353668, 0.9850636549546773, 1.5051540171802662, -0.8655447961985662, -2.177897790250047, 0.6864072972646966, -1.2408238023658449,

In [7]:
#headers of parametrs for normalized periodic table
h_edited_row=[]
for i in range(len(params_order)):
    h_edited_row.append(h_ptrow[i])
    
for i in range(len(one_hot_params_order)):
    h_edited_row.append(h_ptrow[i]) 

print("tsese valuse must be equal")
print(len(h_edited_row))
print(num_params)

print("selected parametrs")
print(h_edited_row)


tsese valuse must be equal
16
16
selected parametrs
['AtomicNumber', 'Element', 'Symbol', 'AtomicMass', 'NumberofNeutrons', 'NumberofProtons', 'NumberofElectrons', 'Period', 'Group', 'Phase', 'Radioactive', 'Natural', 'Metal', 'AtomicNumber', 'Element', 'Symbol']


In [8]:
#writing of normalized parametrs 
#If you make dataset just for training you can skip this step


PTECUTEDwriter.writerow(h_edited_row)
PTECwriter.flush()
    


for i in range(len(mean_clean_pt_list)):
    PTECUTEDwriter.writerow(mean_clean_pt_list[i])
    PTECwriter.flush()

PTECwriter.close() 


In [8]:
#readig strings from superconductors dataset
mrow = next(materialreader)
counter=0
headers=[]

while(counter<21263):
    counter=counter+1
    mrow = next(materialreader)
    headers.append(mrow)

In [9]:
#processing strings with formulas
counter=0
edited_table=[]
edited_test_table=[]
edited_train_table=[]
np.random.shuffle(headers)#it will mix all dataset

#counter for elements in formulas
arr_counter=[]
for i in range(86):
    arr_counter.append(0)


while(counter<21263):#21263
    current_row=[]
    for i in range(86):#check every element in  formula
        if float(headers[counter][i])!=0:
            arr_counter[i]=arr_counter[i]+1
            str_form=str(float(headers[counter][i]))+" "#add coefficient of element into processed data
            
            for j in range(num_params):# add parametrs of element into processed data
                str_form=str_form+str(mean_clean_pt_list[i][j])+" "# clean_pt_list or mean_clean_pt_list
            str_form = str_form[:-1]
            
            current_row.append(str_form)
    current_row.append(str(headers[counter][86]))#add critical temperature into processed data
    edited_table.append(current_row)
    
    if counter%10==0:
        edited_test_table.append(current_row)
    else:
        edited_train_table.append(current_row)
    counter=counter+1


print(len(edited_test_table))
print(len(edited_train_table))


#86 crit temp

2127
19136


In [11]:
#subsets will be mixed and duplicated 5 times. it will remove batch specific.
temp_min=0#you can create subset for superconductors with different temperature limits     
temp_max=999999#you can create subset for superconductors with different temperature limits     
for i in range (5):
    counter=0
    np.random.shuffle(edited_test_table)#it will mix all subset in every iteration
    while(counter<len(edited_test_table)):
        last_ctemp=edited_test_table[counter][-1]
        new_mixed_element_row = edited_test_table[counter][:-1]
        #np.random.shuffle(new_mixed_element_row)#you can try mix position of element. in my experiments MAE value was 6-8 degree
        new_mixed_element_row.append(last_ctemp)
        if float(last_ctemp)>temp_min and float(last_ctemp)<temp_max:
            testwriter.writerow(new_mixed_element_row)
            testcswriter.flush()
        counter=counter+1
        
for i in range (5):
    counter=0
    np.random.shuffle(edited_train_table)
    while(counter<len(edited_train_table)):
        last_ctemp=edited_train_table[counter][-1]
        new_mixed_element_row = edited_train_table[counter][:-1]
        #np.random.shuffle(new_mixed_element_row)#you can try mix position of element. in my experiments MAE value was 6-8 degree
        new_mixed_element_row.append(last_ctemp)
        if float(last_ctemp)>temp_min and float(last_ctemp)<temp_max:
            trainwriter.writerow(new_mixed_element_row)
            traincswriter.flush()
        counter=counter+1
        

In [12]:

cs_materials.close() 
traincswriter.close()
testcswriter.close()
cs_pt.close()